## ■ ORM 이란
- 기존의 sqlite 라이브러리는 sql문법에 의존해서 파이썬코드로 sql을 동작했었다.
- **sql문법을 전부 파이썬 객체로 맵핑하여 그냥 파이썬 객체의 인스턴스, 메소드를 통해서 구현할 수 있다면 훨씬 더 체계적이고 가독성이 좋을 것에서 시작**
- **그러한 객체와 기능의 파이썬으로의 MAPPING을 바로 ORM이라고 함.**

## ■ ORM 장점 예시

### (1) SQL Table
    CREATE TABLE articles (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        content TEXT NOT NULL,
    );
- SQL Table을 만듬

### (2-1) 데이터가 저장시 (리스트나 딕셔너리라면)
    articles1 = [
        [id1, 'title1입니다', '내용1입니다']
        [id2, 'title2입니다', '내용2입니다']
    ]
    
    articles2 = [
        {'id':1, "title":"제목1","content":"내용1"}
        {'id':2, "title":"제목2","content":"내용2"}
    ]
- 위와 같은 리스트 형태나, 딕셔너리 형태가 될 것임

### (2-2) 데이터가 저장시 Object라면 (객체 class)
    class Article:
    def __init__(self, id, title, content):
        self.id=id
        self.title=title
        self.content=content
        
    a1 = Article(id1,'제목1','내용1')
    a2 = Article(id2,'제목2','내용2')

- 위와 같은 식으로 저장되서 a1.title 식으로 데이터를 갖고 올 수 있을 것.
- 또한 가져온 데이터에 대한 메소드를 수행할 수 있을 것. (클래스니까)

#### ==> 즉 sqlite 라이브러리의 connect 힘을 빌리지 않고도 데이터가 스스로 소멸하거나, 변경 할 수 있다는 점. 이것외의 장점은 되게 많다고 함.

## ■ SQL Alchamy
- Database와 연결하기 위해 사용할 수 있는 라이브러리
- SQLAlchemy는 ORM(Object Relational Mapper)로 관계형 데이터베이스의 테이블들을 프로그래밍 언어의 클래스로 표현할 수 있게 해줌.
- 즉, 클래스(class)를 사용해서 DB TABLE들을 표현하고 저장, 읽기, 업데이트, 삭제 등을 할 수 있음.

### ■ SQL Alchamy 예제 - ORM 적용 예시
    class Flight(db.Model):
        __tablename__ = "flights"
        id = db.Column(db.Integer, primary_key=True)
        origin = db.Column(db.String, nullable=False)
        destination = db.Column(db.String, nullable=False)    
        duration = db.Column(db.Integer, nullable=False)   
        
### ■ Flask 관련
- app = Flask(__name__) # Flask 객체를 app에 할당
- app.config['SQLALCHEMY_DATABASE_URI'] # 내가 사용할 DATABASE 의 URI를 넣는다.
- app.config['SQLALCHEMY_COMMIT_ON_TEARDOWN'] = True # Tear down => 사용자가 웹사이트에 요청을 하고서, 서버에서 사용자에게 원하는 정보를 전달햇을때 / COMMIT => 사용자가 웹사이트에 요청한걸 사용자에게 주었을 때, 그러한 동작들을 쌓아놧다가 한번에 COMMIT 하면 DATABASE 에 직접적으로 반영됨


### ■ db관련
- sqlite DB사용 따로 설치 필요없이 파일로 만들어서 사용 가능
- db = SQLAlchemy(app) # db에 대한 최상위 변수
- class Test(db.Model): # db.Model을 상속받은 class 만들어줌. 
- **db.create_all() # 이렇게 만든 테이블을 다 생성해봐라**
- db.Model을 상속받으면 db.Column()이라는 메소드를 사용가능
- 항상 database에 직접 반영할때는 commit() 을 해주어야함.

In [23]:

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask import request

import os # 절대 경로 사용

basedir = os.path.abspath(os.path.dirname('__file__'))
dbfile = os.path.join(basedir, 'db.sqlite')

# Flask 객체를 app에 할당
app = Flask(__name__)

### SQLlite 설정
# sqlite DB사용 따로 설치 필요없이 파일로 만들어서 사용 가능
# 내가 사용할 DATABASE 의 URI를 넣는다. / SQLITE나 MYSQL 에 따라서 다르다.
# /// 3개면 상대경로 //// 4개면 절대경로
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + dbfile

# Tear down => 사용자가 웹사이트에 요청을 하고서, 서버에서 사용자에게 원하는 정보를 전달햇을때
# COMMIT => 사용자가 웹사이트에 요청한걸 사용자에게 주었을 때, 그러한 동작들을 쌓아놧다가 한번에 COMMIT 하면 DATABASE 에 직접적으로 반영됨
# 따라서 COMMIT 은 그러한 작업들을 데이터베이스에 직접적으로 반영하는 것 !
app.config['SQLALCHEMY_COMMIT_ON_TEARDOWN'] = True
app.config['SQLALCHEMY_TRACK_NOTIFICATIONS'] = False # 변경에 대한 알림 안받기

# db에 대한 최상위 변수
db = SQLAlchemy(app)
db.init_app(app)

# Create Table
# 하나의 모델 (MVC의 M)
# db.Model을 상속받으면 db.Column()이라는 메소드를 사용가능함.

class Article(db.Model):
    __table_name = 'article_table' # table 명
    db_id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(32), unique=True)

# db 생성
db.create_all()


# app 객체를 이용해 라우팅 경로를 설정
# 해당 라우팅 경로로 요청이 올 때 실행할 함수를 바로 밑에 작성필요

@app.route('/')
def hello():
    return '<h1>Hello world!</h1>'

if __name__ == "__main__":
    host_addr = "127.0.0.1"
    port_num = "8080"
    app.run(host=host_addr, port=port_num)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


c:\users\urse\anaconda3\envs\untitled\lib\site-packages\flask_sqlalchemy\__init__.py:834: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
c:\users\urse\anaconda3\envs\untitled\lib\site-packages\flask_sqlalchemy\__init__.py:834: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)


## INSERT INTO

In [ ]:
""" ORM을 이용하여 데이터를 저장한다. """
db_id = request.args.get('db_id')
name = request.args.get('name')

# insert
a = Article(db_id=db_id, name=name) # Article이라는 객체 생성
db.session.add(a) # 생성한 객체를 디비에 넣어라.
db.session.commit() # commit

## SELECT
- python sqlite 라이브러리를 이용하면 db연결하고, cursor만들고, execute하고 fetch해야하는데 이건 한줄로 다 처리할 수 있다.

In [ ]:
article = Article.query.all()
# SELECT * FROM articles와 동일함
# 이렇게 뽑힌 것들은 객체들의 리스트가 된다.
# [<article1>, <article2>, ...]
# 따라서 객체의 원소를 참조하려면 멤버변수를 통해서 들고와야한다!

## WHERE
- filter_by라는 메서드를 통해서 제어를 할 수 있다.
- 우리가 많이 쓰는 where문은 바로 id를 통해서 제어하는 방식인데, 이건 더 간단한 방식으로 구현되어있다.
- get 메서드도 가능

In [ ]:
a = Article.query.filter_by(name="이름인가요?").all()

a = Article.query.filter_by(id=28).first()
# SELECT * FROM articles WHERE id=28 LIMIT 1과 동일한 문법
a = Article.query.get(28) # 이렇게 간단하게도 줄여쓸 수 있다!

## DELETE
- 삭제할 객체를 찾는다 -> 객체를 제거한다

In [ ]:
a = Article.query.get(28)
db.session.delete(a)
db.session.commit()
# 삭제할 객체를 찾는다 -> 객체를 제거한다

## UPDATE

In [ ]:
a.name = "Good Title!"
# 각 row가 인스턴스이기 때문에 클래스의 멤버변수에 직접 접근해서 바꾸면 됨.
db.session.commit() # 반드시 커밋!

## Count

In [6]:
#SELECT COUNT(*) FROM articles
Article.query.count()

NameError: name 'Article' is not defined

## MAX

In [ ]:
a = Article.query.order_by(Article.id.desc()).first()

## 참고문헌
- https://dongchans.github.io/2019/28/ (ORM 과 SQLAlchemy란)